In [1]:
import keras
from keras.models import Sequential
#from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils
import cv2
import math
import os
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
train = pd.read_csv(r'D:\2020Spring\636_project\data\train_new.csv')
train.head()
#print(train)

,image,class
0,S012C001P007R001A001_rgb.avi_frame0.jpg,Others
1,S012C001P007R001A001_rgb.avi_frame1.jpg,Others
2,S012C001P007R001A001_rgb.avi_frame10.jpg,Others
3,S012C001P007R001A001_rgb.avi_frame11.jpg,Others
4,S012C001P007R001A001_rgb.avi_frame12.jpg,Others


In [3]:
#creating an empty list
#train_image = []
#train_image_temp = []
X = []
i_count = train.shape[0]
i_count = int(i_count/15)
print(i_count)
# for loop to read and store frames
for i in tqdm(range(i_count)):
#for i in range(150):
    train_image = []
    train_image_temp = []
    for j in range(15):
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(r'D:\2020Spring\636_project\data\train_1/'+train['image'][i*15+j])
        #print(i*15+j)
        #print(train['image'][i*15+j])
        # converting it to array
        img = image.img_to_array(img)
        # appending the image to the train_image list
        train_image_temp.append(img)
        #print(len(train_image_temp))
    #train_image.append(train_image_temp)
    X.append(np.array(train_image_temp,'float16'))
#print(train_image[2])
#train_image.dtype = 'float16'
# converting the list to numpy array
#X = np.array(train_image)
X_train = np.array(X,dtype='float16')
# shape of the array
X_train.shape

100


100%|██████████| 100/100 [00:04<00:00, 20.54it/s]


(100, 15, 112, 112, 3)

In [4]:
print(len(X_train))

100


In [5]:
label = np.zeros(len(X_train),dtype = int)
count = 5
for i in tqdm(range(len(X_train))):
    if (i == count and i < 48):
        label[i] = 1
        count += 6
    if  (i == 48 or i == 54):
        label[i] = 1
    if (i > 59):
        label[i] = 1
print(label)

100%|██████████| 100/100 [00:00<?, ?it/s]


[0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [6]:
'''
label = np.zeros(len(X_train),dtype = int)
count = 40
for i in tqdm(range(len(X_train))):
    if (i == count):
        label[i] = 1
        count += 60
#print(label[101])

num_1 = train['image']
print(num_1)
label = np.zeros(len(X_train),dtype = int)
for i in tqdm(range(len(X_train))):
    num = num_1[i].split('\\',5)[5]
    # creating the image name
    #print(images[i].split('_rgb')[0].split('A')[1])
    if (num[i].split('_rgb')[0].split('A')[1] == '041'):
        label.append('Cough')
    # creating the class of image
    else:
        label.append('Others')
'''

"\nlabel = np.zeros(len(X_train),dtype = int)\ncount = 40\nfor i in tqdm(range(len(X_train))):\n    if (i == count):\n        label[i] = 1\n        count += 60\n#print(label[101])\n\nnum_1 = train['image']\nprint(num_1)\nlabel = np.zeros(len(X_train),dtype = int)\nfor i in tqdm(range(len(X_train))):\n    num = num_1[i].split('\\',5)[5]\n    # creating the image name\n    #print(images[i].split('_rgb')[0].split('A')[1])\n    if (num[i].split('_rgb')[0].split('A')[1] == '041'):\n        label.append('Cough')\n    # creating the class of image\n    else:\n        label.append('Others')\n"

In [7]:
# creating the training and validation set
X_train = X_train.astype('float32')
X_train -= np.mean(X_train)
X_train /=np.max(X_train)

label = np_utils.to_categorical(label, 2)
X_train, X_test, y_train, y_test = train_test_split(X_train, label, random_state=42, test_size=0.2, stratify = label)

In [8]:
#print(y_train, y_test)

In [9]:
model = Sequential()
model.add(Convolution3D(32, 5, 5, 5, padding='same', name='conv1', dim_ordering='tf', input_shape=(15,112,112,3), activation='relu'))

model.add(MaxPooling3D(pool_size=(3, 3, 3)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(2,init='normal'))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

D:\Software\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (5, 5, 5), padding="same", name="conv1", input_shape=(15, 112, ..., activation="relu")`
  
D:\Software\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="normal")`
  # Remove the CWD from sys.path while we load stuff.
D:\Software\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`
  


In [10]:
hist = model.fit(X_train, y_train, validation_data=(X_test,y_test),
          batch_size = 16, epochs = 15, shuffle=True)


Train on 80 samples, validate on 20 samples
Epoch 1/15
80/80 [==============================] - ETA: 43s - loss: 1.2344 - accuracy: 0.625 - ETA: 32s - loss: 5.5855 - accuracy: 0.593 - ETA: 21s - loss: 46.9553 - accuracy: 0.52 - ETA: 10s - loss: 42.0002 - accuracy: 0.53 - 55s 683ms/step - loss: 38.1238 - accuracy: 0.5500 - val_loss: 1.6010 - val_accuracy: 0.7500
Epoch 2/15
80/80 [==============================] - ETA: 42s - loss: 4.6704 - accuracy: 0.625 - ETA: 31s - loss: 3.0842 - accuracy: 0.687 - ETA: 21s - loss: 5.1527 - accuracy: 0.604 - ETA: 10s - loss: 5.0034 - accuracy: 0.609 - 54s 676ms/step - loss: 4.4904 - accuracy: 0.6000 - val_loss: 0.9266 - val_accuracy: 0.7500
Epoch 3/15
80/80 [==============================] - ETA: 43s - loss: 1.4419 - accuracy: 0.687 - ETA: 32s - loss: 1.3586 - accuracy: 0.750 - ETA: 21s - loss: 1.4590 - accuracy: 0.750 - ETA: 10s - loss: 2.2556 - accuracy: 0.734 - 55s 682ms/step - loss: 1.9730 - accuracy: 0.7250 - val_loss: 0.4352 - val_accuracy: 0.750

In [11]:
score = model.evaluate(X_test, y_test, batch_size=32)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

20/20 [==============================] - 1s 75ms/step


In [12]:
print(score)

[0.5563778281211853, 0.5]


In [13]:
'''
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accrucy']
val_acc=hist.history['val_acc']
xc=range(100)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
'''

"\ntrain_loss=hist.history['loss']\nval_loss=hist.history['val_loss']\ntrain_acc=hist.history['accrucy']\nval_acc=hist.history['val_acc']\nxc=range(100)\n\nplt.figure(1,figsize=(7,5))\nplt.plot(xc,train_loss)\nplt.plot(xc,val_loss)\nplt.xlabel('num of Epochs')\nplt.ylabel('loss')\nplt.title('train_loss vs val_loss')\nplt.grid(True)\nplt.legend(['train','val'])\nplt.style.available # use bmh, classic,ggplot for big pictures\nplt.style.use(['classic'])\n\nplt.figure(2,figsize=(7,5))\nplt.plot(xc,train_acc)\nplt.plot(xc,val_acc)\nplt.xlabel('num of Epochs')\nplt.ylabel('accuracy')\nplt.title('train_acc vs val_acc')\nplt.grid(True)\nplt.legend(['train','val'],loc=4)\n#print plt.style.available # use bmh, classic,ggplot for big pictures\nplt.style.use(['classic'])\n"

In [16]:
# open the .txt file which have names of test videos
f = open(r"D:\2020Spring\636_project\data\test_list.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:]
#print(test)

In [17]:
# storing the frames from test videos
Test_array = []
for i in tqdm(range(test.shape[0])):
#for i in range(1):
    test_image_temp = []
    count = 0
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(r'D:\2020Spring\636_project\data\test_data1/' + videoFile)   # capturing the video from the given path
    frameCount = cap.get(7) #frame counts
    x = math.floor(frameCount / 15)
    #print([frameCount,x])
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (count == 15):
            break
        if (frameId == (x * count)):
            # storing the frames in a new folder named train_1
            frame=cv2.resize(frame,(112,112),interpolation=cv2.INTER_AREA)
            #filename =r'D:\2020Spring\636_project\data\test_1/' + videoFile +"_frame%d.jpg" % count;count+=1
            #cv2.imwrite(filename, frame)
            frame = image.img_to_array(frame)
            # appending the image to the train_image list
            test_image_temp.append(frame)
            #print(len(test_image_temp))
            count+=1
    cap.release()
    Test_array.append(np.array(test_image_temp,dtype='float16'))
Test_array_f = np.array(Test_array,dtype='float16')
Test_array_f = Test_array_f.astype('float32')
Test_array_f -= np.mean(Test_array_f)
Test_array_f /=np.max(Test_array_f)
#Test_array_f[0][:][:][:][:] = Test_array[:][:][:][:]
print(Test_array_f.shape)
#make prediction
prediction = model.predict_classes(Test_array_f)
print(prediction)

100%|██████████| 960/960 [09:30<00:00,  1.68it/s]


(960, 15, 112, 112, 3)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1